# LUNAR LANDER

El objetivo del juego es simple (¡pero aterrizar no lo es!): ¡aterrizar la nave espacial sana y salva en la plataforma designada! ¡Prepárate para un aterrizaje suave y heroico! 🚀🌕


## Reglas
En cada momento del juego, ganas o pierdes puntos (recompensa) dependiendo de cómo te vaya:

Aterrizaje y velocidad: Ganas puntos si te acercas a la zona de aterrizaje y vas despacio. Pierdes puntos si te alejas o vas muy rápido.

Inclinación: Pierdes puntos si la nave está muy inclinada. ¡Tienes que mantenerla lo más horizontal posible!
Patas en el suelo: Ganas 10 puntos por cada pata que toca el suelo en la zona de aterrizaje.

Motores: Pierdes puntos por usar los motores: un poquito por los motores laterales y más por el motor principal. ¡Hay que usarlos con cuidado!

Final del juego: Si te estrellas, pierdes 100 puntos. Si aterrizas suavemente en la plataforma, ¡ganas 100 puntos extra!

Para considerar que has tenido éxito en un intento (episodio), ¡necesitas conseguir al menos 200 puntos en total!

## Instalacion de librerias

In [1]:
# Permite conectar codigo en C, C++ con Python
# Requerido por box2d
!pip install -q swig

# Gymnasium provee entornos de simulacion, controles y califica resultado
!pip install -q "gymnasium[classic-control]"
!pip install -q gymnasium[box2d]

# Para grabar y reproducir video
# !pip install moviepy
!pip install -q pyvirtualdisplay

# Agente DQN (Deep Q-learning), al que entrenaremos
!pip install -q stable-baselines3

## Variables globales

In [2]:
ENV_NAME = "LunarLander-v3" # Nombre del entorno
VIDEO_FOLDER = "./video_prueba_de_vuelo" # En esta carpeta se guardaran los videos del test de vuelo
EPISODES = 1 # Numero de episodios a grabar en la prueba de vuelo, se tratara de seleccionar el mejor
LOG_DIR = "./tmp/dqn_lunar" # Carpeta donde se guardarán los registros de entrenamiento (logs)

## Entrenando el modelo

In [3]:
# ==============================================================================
# ENTRENAMIENTO DE UN AGENTE DQN (Stable-Baselines3)
# ==============================================================================

# Gymnasium provee el entorno, controles y evalua el resultado
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import os
# import moviepy.editor as mp # Importamos MoviePy


# Agente DQN, al que entrenaremos
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor


# --- Preparación para el entrenamiento ---
# La grabación de video solo debe hacerse después del entrenamiento o en un ambiente separado.
# Para entrenar, usaremos una versión simple del ambiente sin el wrapper de video.

os.makedirs(LOG_DIR, exist_ok=True)

# Crear el ambiente para el entrenamiento (usando Monitor para guardar logs)
env_train = gym.make(
    ENV_NAME,
    continuous=False,
    gravity=-10,
    enable_wind=False,
    wind_power=15.0,
    turbulence_power=1.5
)
env_train = Monitor(env_train, LOG_DIR)

# Stable-Baselines3 funciona mejor con entornos vectorizados
env_train_vec = make_vec_env(lambda: env_train, n_envs=1)


# --- Creación del Modelo DQN ---
# DQN es un algoritmo de Q-Learning profundo, ideal para ambientes discretos (como LunarLander-v3)
model = DQN(
    "MlpPolicy",         # Tipo de red neuronal (Multi-layer perceptron)
    env_train_vec,       # El ambiente de entrenamiento
    learning_rate=0.0001,  # Tasa de aprendizaje (0.00001 y 0.001)
    buffer_size=10000,   # (10000 - 50000)
    learning_starts=5000,# (1000 - 10000)
    batch_size=64,       # Puede ser [32, 64, 128]
    gamma=0.99,          # Factor de descuento (0.90 - 0.99) menor=quiero recompensas rapido, mayor=espera recompensas mayores siendo mas cuidadoso
    verbose=1,           # Mostrar el progreso del entrenamiento
    tensorboard_log=LOG_DIR
)

# --- Bucle de Aprendizaje ---
# El método .learn() es el núcleo del entrenamiento de RL.
# Entrenaremos por (50,000 - 200,000) pasos (timesteps). Esto tomará unos minutos en Colab.
TIMESTEPS = 100_000
print(f"\n--- INICIANDO ENTRENAMIENTO DQN por {TIMESTEPS} pasos ---")

# Entrenar!!
model.learn(
    total_timesteps=TIMESTEPS,
    log_interval=100
)

print("\n--- ENTRENAMIENTO FINALIZADO. Modelo entrenado guardado. ---")
model.save("modelo_nave_entrenada") # Guarda el modelo entrenado
env_train.close()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203

Using cpu device


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



--- INICIANDO ENTRENAMIENTO DQN por 100000 pasos ---
Logging to ./tmp/dqn_lunar/DQN_3
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 138      |
|    ep_rew_mean      | -240     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 100      |
|    fps              | 1223     |
|    time_elapsed     | 11       |
|    total_timesteps  | 13784    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.14     |
|    n_updates        | 2195     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 329      |
|    ep_rew_mean      | -88.3    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 200      |
|    fps              | 871      |
|    time_elapsed     | 53       |
|    total_timesteps  | 46677    |
| train/              |          |
|  

## Prueba de Vuelo

In [4]:
# ==============================================================================
# 4. PRUEBA DE VUELO Y GRABAR EL VIDEO
# ==============================================================================
from IPython.display import HTML
from base64 import b64encode
import glob
import io
from pyvirtualdisplay import Display

# Google collab tiene dependencias core deprecadas
import warnings
warnings.filterwarnings('ignore')

# 1. Configurar la Pantalla Virtual (Necesario para Colab/Jupyter sin GUI)
print("\n--- Configurando Pantalla Virtual ---")
try:
    display = Display(visible=0, size=(640, 480))
    display.start()
    print("Pantalla virtual iniciada.")
except Exception as e:
    print(f"Advertencia al iniciar pyvirtualdisplay: {e}. Continuaremos.")

# 2. Crear un nuevo ambiente con el wrapper RecordVideo
# Creamos la carpeta de video si no existe
os.makedirs(VIDEO_FOLDER, exist_ok=True)
print(f"Grabando {EPISODES} episodio(s) en la carpeta: {VIDEO_FOLDER}")

# Creamos el ambiente de test con el wrapper de video
env_test = gym.make(
    ENV_NAME,
    continuous=False,
    gravity=-10,
    enable_wind=False,
    wind_power=15.0,
    turbulence_power=0.1,
    render_mode="rgb_array"
)
# El wrapper de RecordVideo debe ser el que envuelve al ambiente base
env_test_video = RecordVideo(
    env_test,
    video_folder=VIDEO_FOLDER,
    episode_trigger=lambda x: x == 0, # Graba solo el primer episodio
    name_prefix="prueba_de_vuelo"
)

# 3. Cargar el modelo entrenado y ejecutar un episodio
# Cargamos el modelo que acabamos de entrenar y guardar
model = DQN.load("modelo_nave_entrenada", env=env_test_video)

obs, info = env_test_video.reset()
done = False
truncated = False
while not (done or truncated):
    # El modelo determina la acción
    action, _ = model.predict(obs, deterministic=True)
    # Ejecutamos la acción
    obs, reward, done, truncated, info = env_test_video.step(action)

env_test_video.close()
print("\n--- Grabación del video finalizada. ---")


--- Configurando Pantalla Virtual ---
Pantalla virtual iniciada.
Grabando 1 episodio(s) en la carpeta: ./video_prueba_de_vuelo
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

--- Grabación del video finalizada. ---


## Reproducir Video de la prueba

In [5]:
# ==============================================================================
# 5. CARGAR Y REPRODUCIR EL VIDEO DE LA PRUEBA DE VUELO
# ==============================================================================
import os
import glob
import io
from IPython.display import HTML, display
from base64 import b64encode

# 1. Función para codificar y mostrar un video usando Base64
def display_encoded_video(video_path):
    """Codifica un video a Base64 y lo muestra en un Jupyter/Colab notebook."""
    print(f"Mostrando: {video_path}")

    try:
        # Abrir y codificar el archivo
        with io.open(video_path, 'rb') as f:
            video_bytes = f.read()
        video_encoded = b64encode(video_bytes).decode()

        # Crear y mostrar el tag de video HTML
        html_tag = f"""
        <video width="600" controls autoplay>
            <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
            Tu navegador no soporta el tag de video.
        </video>
        <p>--------------------------------------------------</p>
        """
        display(HTML(html_tag))

    except Exception as e:
        print(f"❌ ERROR al procesar o mostrar el video {video_path}: {e}")
        print("Esto podría ser por un archivo muy grande.")


# 2. Buscar todos los archivos .mp4 en la carpeta
# Ordenamos por fecha de creación para verlos en orden de grabación
list_of_files = sorted(
    glob.glob(os.path.join(VIDEO_FOLDER, "*.mp4")),
    key=os.path.getctime
)

# 3. Iterar y mostrar cada video
if list_of_files:
    print(f"✅ Se encontraron {len(list_of_files)} videos para reproducir.")
    for video_file in list_of_files:
        display_encoded_video(video_file)
else:
    print(f"❌ No se encontró ningún archivo de video MP4 en {VIDEO_FOLDER}.")

✅ Se encontraron 1 videos para reproducir.
Mostrando: ./video_prueba_de_vuelo/prueba_de_vuelo-episode-0.mp4


## Puntaje de la prueba

In [6]:
# ----------------------------------------------------------------------
# CALIFICACION DEL ENTRENAMIENTO
# ----------------------------------------------------------------------

# Asume que estas variables ya han sido actualizadas por env_test_video.step()
# reward, done, truncated, info


# Imprimir cada variable en una línea separada
print(f"Reward (Recompensa): {reward:.2f}")
print(f"Done (Logro Completar?): {done}")
print(f"Truncated (Tuvo que interrumpirse?): {truncated}")
print(f"Info (Información): {info}")

Reward (Recompensa): -100.00
Done (Logro Completar?): True
Truncated (Tuvo que interrumpirse?): False
Info (Información): {}
